# k-Means

Your task is to implement the k-means algorithm.

In [ ]:
%pylab inline
from sklearn import datasets

## Artificial Dataset

We will work with an artificial 2D dataset that contains 3 clusters.

In [ ]:
n_samples = 200
random_state = np.random.RandomState(10)
X, _ = datasets.make_blobs(n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state)

plt.scatter(X[:, 0], X[:, 1])
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.title("Artificial dataset")
plt.tight_layout()

## k-Means Algorithm

This is the outline of the k-means algorithm. You will implement its steps in the following sections.

In [ ]:
class KMeans:
    """k-means clustering algorithm.

    Parameters
    ----------
    k : int
        Number of clusters

    rng_seed : int, optional (default: 0)
        Seed of random number generator
    """
    def __init__(self, k, rng_seed=0):
        self.k = k
        self.rng_seed = rng_seed

    def fit(self, X):
        """Run k-means clustering on dataset and return cluster centers for given samples.

        Parameters
        ----------
        X : array, shape (n_samples, n_features)
            Dataset

        Returns
        -------
        nearest_centers : array, shape (n_samples,)
            Indices of nearest center for each sample / cluster index
        """
        self.means = randomly_initialize_cluster_centers(X, self.k, self.rng_seed)
        while True:
            nearest_centers = assign_all_datapoints_to_their_nearest_center(X, self.means)
            last_means = self.means
            self.means = relocate_centers_to_mean_of_assigned_datapoints(X, last_means, nearest_centers)
            if converged(last_means, self.means):
                break
        return self

### (1) Randomly initialize cluster centers:

Make initial guesses for each cluster mean (center) $\boldsymbol{m}_1, \boldsymbol{m}_2, \ldots, \boldsymbol{m}_k$

**What is a good guess?**

In [ ]:
def randomly_initialize_cluster_centers(X, k, rng_seed):
    """Randomly initialize cluster centers.

    Parameters
    ----------
    X : array, shape (n_samples, n_features)
        Dataset

    k : int
        Number of clusters

    Returns
    -------
    means : array, shape (k, n_features)
        Initial centers
    """
    rng = np.random.default_rng(rng_seed)
    indices_of_initial_means = rng.integers(0, len(X), k)
    means = X[indices_of_initial_means]
    return means

### (2) Assign all datapoints to their nearest center
Partition the observations based on the distances to cluster means: $$C_i^l = \{\boldsymbol{x}_p: ||\boldsymbol{x}_p - \boldsymbol{m}_i^l||_2 \leq || \boldsymbol{x}_p - \boldsymbol{m}_i^j||_2\, \forall 1 \leq j \leq k\}$$

<center><img src="kmeans.png" width=600px /></center>

In [ ]:
def assign_all_datapoints_to_their_nearest_center(X, means):
    """Assign all datapoints to their nearest center.

    Parameters
    ----------
    X : array, shape (n_samples, n_features)
        Dataset

    means : array, shape (k, n_features)
        Cluster centers

    Returns
    -------
    nearest_centers : array, shape (n_samples,)
        Indices of nearest cluster centers per sample
    """
    nearest_centers = np.zeros(len(X), dtype=int)
    for i in range(len(X)):
        distances_to_means = np.linalg.norm(X[i] - means, axis=1)
        nearest_centers[i] = np.argmin(distances_to_means)
    return nearest_centers

### (3) Relocate centers to mean of assigned datapoints

Replace $\boldsymbol{m}_i^l$ with the mean of all the samples in cluster $l$, i.e., $$\boldsymbol{m}^l_{i+1} = \frac{1}{|C^l_i|} \sum_{\boldsymbol{x}_p \in C^l_i} \boldsymbol{x}_p$$

In [ ]:
def relocate_centers_to_mean_of_assigned_datapoints(X, means, nearest_centers):
    """Relocate centers to mean of assigned datapoints.

    Parameters
    ----------
    X : array, shape (n_samples, n_features)
        Dataset

    means : array, shape (k, n_features)
        Current cluster centers

    nearest_centers : array, shape (n_samples,)
        Indices of nearest cluster centers per sample

    Returns
    -------
    new_means : array, shape (k, n_features)
        New cluster centers
    """
    new_means = np.empty_like(means)
    for l in range(len(means)):
        X_cluster_l = X[np.where(nearest_centers == l)]
        new_means[l] = np.mean(X_cluster_l, axis=0)
    return new_means

### (4) Repeat 2. and 3. until convergence

In [ ]:
def converged(last_means, means):
    """Check if k-means converged.

    Parameters
    ----------
    last_means : array, shape (k, n_features)
        Last cluster centers

    means : array, shape (k, n_features)
        Current cluster centers

    Returns
    -------
    converged : bool
        Did k-means converge?
    """
    diff_between_means = np.linalg.norm(means - last_means)
    return diff_between_means < 1e-10

## Let's Use It!

In [ ]:
kmeans = KMeans(k=3, rng_seed=1)
kmeans.fit(X)

In [ ]:
centers = kmeans.means
clusters = assign_all_datapoints_to_their_nearest_center(X, centers)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=clusters, label="Dataset (color indicates cluster)")
plt.scatter(centers[:, 0], centers[:, 1], c="r", label="Cluster centers", s=200)
plt.legend()
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.title("Clusters")
plt.tight_layout()